MultiModal RAG App for Video Processing With LlamaIndex and LanceDB
1. llamaindex framework
2. Lancedb Vector DataBase
3. LLM MultiModAl GPT-4V or Google-gemini-pro-vision

Steps Need to follow:

1. Download video from YouTube, process and store it.
2. Build Multi-Modal index and vector store for both texts and images.
3. Retrieve relevant images and context, use both to augment the prompt.
4. Using GPT4V for reasoning the correlations between the input query and augmented data and generating final response.

In [57]:
%pip install llama-index-vector-stores-lancedb
%pip install gemini
%pip install llama-index-vector-stores-lancedb
%pip install llama-index-embeddings-gemini  # Replace with Gemini-specific embeddings
%pip install llama-index-readers-file
%pip install llama-index-multi-modal-llms-openai
%pip install llama-index-embeddings-clip
%pip install git+https://github.com/openai/CLIP.git
!pip install llama-index-readers-file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 105.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.6 MB/s eta 

In [2]:
%pip install llama_index
%pip install -U openai-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=b6afb2316a163a5c9ae8f709211d5b38fa4e9d958383384f680ed8a99f4d43b4
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


ffmpeg-library enables you to use FFmpeg in Python to manipulate various media files for different purposes like building comprehensive multimedia applications, preprocessing media files.

MoviePy is a Python library for video editing, enabling cutting, concatenations, title insertions, video compositing, and effects like animations or color grading.

Pytube is a Python library used for downloading videos from YouTube. It supports downloading in various formats, resolutions, and also direct audio extraction.

Pydub is a Python library for audio manipulation, enabling easy loading, editing, and exporting of audio files in various formats with minimal code.

The SpeechRecognition library in Python allows you to convert spoken language into text using various engines and APIs, such as Google Speech Recognition, IBM Speech to Text, etc.

SoundFile is a Python library for reading from and writing to audio files, supporting many formats through the libsndfile library, ideal for high-quality audio processing.

FTFY (Fix Text For You) is a Python library that fixes broken Unicode text and mojibake (garbled text due to encoding issues), making text legible again.

OpenAI Whisper is a robust, multilingual speech recognition model developed by OpenAI. It converts speech into text and supports various languages with high accuracy.

pprint is a Python module that provides a capability to "pretty-print" complex data structures in a well-formatted and more readable way than the basic print function.

In [1]:
%pip install lancedb
%pip install moviepy
%pip install pytube
%pip install pydub
%pip install SpeechRecognition
%pip install ffmpeg-python
%pip install soundfile
%pip install torch torchvision
%pip install matplotlib scikit-image
%pip install ftfy regex tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 60.0 MB/s eta 0:00:00


In [2]:
from moviepy.editor import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt

  if event.key is 'enter':



In [71]:
import os
from google.colab import userdata
HUGGING_FACE_KEY=userdata.get('HUGGING_FACE_KEY')
os.environ["HUGGING_FACE_KEY"] = HUGGING_FACE_KEY

In [4]:
import os
print(os.getcwd())

/content


In [23]:
video_url="https://www.youtube.com/watch?v=rijqfllOq6g"

In [31]:
output_video_path = "/content/video_data/"

In [7]:
# from the video i am going to collect images,audio,text
output_folder = "/content/mixed_data/"
output_audio_path = "/content/mixed_data/output_audio.wav"

In [8]:
!mkdir mixed_data

In [9]:
filepath=output_video_path + "input_vid.mp4"
print(filepath)

/content/video_data/input_vid.mp4


In [11]:

from pytube import YouTube
def download_video(url,output_path):
  yt = YouTube(url)
  metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}

  yt.streams.get_highest_resolution().download(
        output_path=output_path, filename="input_vid.mp4"
    )
  return metadata

In [12]:
from moviepy.editor import VideoFileClip
def video_to_images(video_path,output_folder):
  clip=VideoFileClip(video_path)
  clip.write_images_sequence(
      os.path.join(output_folder,"frame%04d.png"),fps=0.2
  )

In [13]:

def video_to_audio(video_path,output_audio_path):
  clip=VideoFileClip(video_path)
  audio=clip.audio
  audio.write_audiofile(output_audio_path)

In [14]:
def audio_to_text(audio_path):
  recognizer=sr.Recognizer()
  audio=sr.AudioFile(audio_path)

  with audio as source:
    audio_data=recognizer.record(source)

    try:

      #recognize the speech
      text = recognizer.recognize_whisper(audio_data)

    except sr.UnknownValueError:
      print("Speech recognition could not understand the audio.")
  return text

In [24]:
video_url

'https://www.youtube.com/watch?v=rijqfllOq6g'

In [25]:
output_video_path

'/content/video_data/'

In [26]:
output_audio_path

'/content/mixed_data/output_audio.wav'

In [27]:
pip install --upgrade pytube


In [37]:
!mkdir video_data

mkdir: cannot create directory ‘video_data’: File exists


In [38]:
video_to_images(filepath,output_folder)




t:  30%|██▉       | 13/44 [05:25<00:02, 13.29it/s, now=None]

t:  30%|██▉       | 13/44 [04:46<00:02, 14.10it/s, now=None]

Moviepy - Writing frames /content/mixed_data/frame%04d.png.





t:   0%|          | 0/44 [00:00<?, ?it/s, now=None]


t:   9%|▉         | 4/44 [00:00<00:01, 22.07it/s, now=None]


t:  16%|█▌        | 7/44 [00:00<00:02, 14.37it/s, now=None]


t:  20%|██        | 9/44 [00:00<00:02, 13.83it/s, now=None]


t:  25%|██▌       | 11/44 [00:00<00:02, 13.20it/s, now=None]


t:  30%|██▉       | 13/44 [00:00<00:02, 14.10it/s, now=None]


t:  34%|███▍      | 15/44 [00:01<00:02, 13.36it/s, now=None]


t:  39%|███▊      | 17/44 [00:01<00:02, 11.68it/s, now=None]


t:  43%|████▎     | 19/44 [00:01<00:02, 11.16it/s, now=None]


t:  48%|████▊     | 21/44 [00:01<00:02, 10.82it/s, now=None]


t:  52%|█████▏    | 23/44 [00:01<00:01, 10.95it/s, now=None]


t:  57%|█████▋    | 25/44 [00:02<00:01, 11.37it/s, now=None]


t:  61%|██████▏   | 27/44 [00:02<00:01, 11.97it/s, now=None]


t:  66%|██████▌   | 29/44 [00:02<00:01, 13.28it/s, now=None]


t:  70%|███████   | 31/44 [00:02<00:01, 12.78it/s, now=None]


t:  75%|███████▌  | 33/44 [00:02<00:01, 10.98it/s, now=None]


t

Moviepy - Done writing frames /content/mixed_data/frame%04d.png.


In [39]:
filepath

'/content/video_data/input_vid.mp4'

In [40]:
video_to_audio(filepath,output_audio_path)




t:  30%|██▉       | 13/44 [05:59<00:02, 13.29it/s, now=None]

t:  30%|██▉       | 13/44 [05:20<00:02, 14.10it/s, now=None]

MoviePy - Writing audio in /content/mixed_data/output_audio.wav





chunk:   0%|          | 0/4763 [00:00<?, ?it/s, now=None]


chunk:   2%|▏         | 79/4763 [00:00<00:06, 762.22it/s, now=None]


chunk:   3%|▎         | 156/4763 [00:00<00:06, 670.89it/s, now=None]


chunk:   7%|▋         | 310/4763 [00:00<00:04, 1035.95it/s, now=None]


chunk:  12%|█▏        | 586/4763 [00:00<00:02, 1681.82it/s, now=None]


chunk:  18%|█▊        | 867/4763 [00:00<00:01, 2061.36it/s, now=None]


chunk:  24%|██▍       | 1150/4763 [00:00<00:01, 2313.42it/s, now=None]


chunk:  30%|██▉       | 1418/4763 [00:00<00:01, 2429.10it/s, now=None]


chunk:  35%|███▌      | 1683/4763 [00:00<00:01, 2477.25it/s, now=None]


chunk:  41%|████      | 1949/4763 [00:00<00:01, 2531.57it/s, now=None]


chunk:  47%|████▋     | 2216/4763 [00:01<00:00, 2573.12it/s, now=None]


chunk:  52%|█████▏    | 2477/4763 [00:01<00:00, 2584.21it/s, now=None]


chunk:  57%|█████▋    | 2737/4763 [00:01<00:00, 2474.85it/s, now=None]


chunk:  63%|██████▎   | 2986/4763 [00:01<00:00, 2427.81it/s, now=None

MoviePy - Done.


In [41]:
text_data=audio_to_text(output_audio_path)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 92.5MiB/s]


In [42]:

text_data

" The correlation coefficient, also called the Pearson correlation, is denoted by R. It measures the strength of a relationship between two variables as a value between negative one and one. A correlation coefficient closer to zero indicates there is no correlation. If it's closer to one, it indicates a strong positive correlation, meaning that when one variable increases, the other proportionally increases. If it's closer to negative one, then it indicates a strong negative correlation, which means as one variable increases, the other proportionally decreases. There are different implementations of the correlation coefficient. When we are talking about a linear relationship, we use the Pearson correlation. It ratios the covariance between the two variables and their product of the standard deviations. However, the covariance is not interesting on its own. Instead, we use it for metrics like the correlation coefficient. And here are the key points. When R approaches zero, it indicates 

In [43]:

with open(output_folder + "output_text.txt", "w") as file:
        file.write(text_data)
print("Text data saved to file")
file.close()

Text data saved to file


In [45]:
os.remove(output_audio_path)
print("Audio file removed")

Audio file removed


In [46]:

#process the video
#image
#text

In [63]:

from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext

In [66]:
pip install --upgrade llama-index


In [70]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1


In [83]:
from llama_index.core import StorageContext
from llama_index.vector_stores.lancedb import LanceDBVectorStore

In [84]:
embedding_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [85]:
# Configure LanceDB vector stores
text_store = LanceDBVectorStore(
    uri="lancedb", table_name="text_collection", embed_model=embedding_model
)
image_store = LanceDBVectorStore(
    uri="lancedb", table_name="image_collection", embed_model=embedding_model
)

In [86]:
storage_context = StorageContext.from_defaults(vector_store=text_store, image_store=image_store)


In [75]:
output_folder

'/content/mixed_data/'

In [76]:
documents=SimpleDirectoryReader(output_folder).load_data()

In [88]:
index = MultiModalVectorStoreIndex.from_documents(documents, storage_context=storage_context)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode

In [ ]:
def retrieve(retriever_engine, query_str):
    retrieval_results = retriever_engine.retrieve(query_str)

    retrieved_image = []
    retrieved_text = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
            retrieved_text.append(res_node.text)

    return retrieved_image, retrieved_text

In [ ]:
query = "Can you tell me waht is Correlation"

In [ ]:
img,text=retrieve(retriever_engine,query)

In [56]:

import matplotlib.pyplot as plt
def plot_images(images_path):
  images_shown = 0
  plt.figure(figsize=(16, 9))
  for img_path in images_path:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 5:
                break

In [ ]:
qa_tmpl_str=(
    "Based on the provided information, including relevant images and retrieved context from the video, \
    accurately and precisely answer the query without any additional prior knowledge.\n"

    "---------------------\n"
    "Context: {context_str}\n"
    "Metadata for video: {metadata_str} \n"

    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

In [ ]:

import json
metadata_str=json.dumps(metadata_vid)

In [ ]:
 query_str="can you tell me what is linear regression and equation of linear regression?"

In [ ]:

context_str = "".join(text)

In [ ]:
image_documents = SimpleDirectoryReader( input_files=img).load_data()


In [ ]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

In [ ]:
openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", api_key=OPENAI_API_TOKEN, max_new_tokens=1500)


In [ ]:

result=openai_mm_llm.complete(
    prompt=qa_tmpl_str.format(
        query_str=query_str,metadata_str=metadata_str
    ),
    image_documents=image_documents,
)

In [ ]:
pprint(result.text)